# EraEx: ColBERT Embedding (Colab GPU)

This notebook generates ColBERT embeddings for music tracks.

**Model**: colbert-ir/colbertv2.0
**Fallback**: sentence-transformers/all-MiniLM-L6-v2

**Requirements**: GPU runtime (T4 or better)

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

from pathlib import Path

try:
    from google.colab import drive
    drive.mount('/content/drive')
    PROJECT_DIR = Path('/content/drive/MyDrive/EraEx')
    print("Running on Google Colab")
except ImportError:
    if Path.cwd().name == 'notebooks':
        PROJECT_DIR = Path.cwd().parent
    else:
        PROJECT_DIR = Path.cwd()
    print(f"Running Locally at {PROJECT_DIR}")

In [ ]:
import subprocess, sys

def run_pip(*args):
    subprocess.check_call([sys.executable, "-m", "pip"] + list(args))

torch_ok = False
try:
    import torch
    torch_ok = True
    print(f"torch {torch.__version__} OK")
except Exception:
    print("torch is broken from a previous install. Reinstalling...")
    run_pip("install", "-q", "--force-reinstall", "torch", "torchvision", "torchaudio")
    print("\n" + "=" * 50)
    print("TORCH FIXED. Restart runtime and re-run all cells.")
    print("Runtime > Restart runtime")
    print("=" * 50)

if torch_ok:
    req_path = PROJECT_DIR / "requirements.txt"
    skip_on_colab = {"torch", "faiss-cpu", "torchvision", "torchaudio"}
    deps = [l.strip() for l in req_path.read_text().splitlines() if l.strip() and l.strip() not in skip_on_colab]
    run_pip("install", "-q", "--no-deps", "sentence-transformers")
    run_pip("install", "-q", *[d for d in deps if d != "sentence-transformers"])
    print("All dependencies installed.")

In [ ]:
import torch
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

In [ ]:
import numpy as np
import polars as pl
import gc
import shutil
from tqdm import tqdm

READY_DIR = PROJECT_DIR / 'data' / 'processed' / 'music_ready'

try:
    from google.colab import drive
    EMBEDDINGS_DIR = Path('/content/embeddings')
    print("Saving embeddings to LOCAL disk (fast)")
except ImportError:
    EMBEDDINGS_DIR = PROJECT_DIR / 'data' / 'embeddings'

EMBEDDINGS_DIR.mkdir(parents=True, exist_ok=True)

YEAR_RANGE = range(2012, 2019)
USE_COLBERT = True
EMBED_CHUNK = 4_000_000

print(f'Project Dir: {PROJECT_DIR}')
print(f'Data Dir: {READY_DIR}\n')

for y in YEAR_RANGE:
    p = READY_DIR / f'year={y}' / 'data.parquet'
    if p.exists():
        rows = pl.scan_parquet(str(p)).select(pl.len()).collect().item()
        print(f'  {y}: {rows:,} rows')
    else:
        print(f'  {y}: NOT FOUND')

In [ ]:
from sentence_transformers import SentenceTransformer

if USE_COLBERT:
    try:
        from transformers import AutoTokenizer, AutoModel, logging
        
        # Suppress HF warnings
        logging.set_verbosity_error()
        import warnings
        warnings.filterwarnings('ignore', category=UserWarning)

        MODEL_NAME = 'colbert-ir/colbertv2.0'
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        
        print(f'Loading {MODEL_NAME}...')
        print("(Note: Ignore 'UNEXPECTED linear.weight' warnings - normal for Dense mode)")
        
        model = AutoModel.from_pretrained(MODEL_NAME, attn_implementation="sdpa")
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        model.to(device)
        if device == 'cuda':
            model.half()
            print("fp16 + SDPA (Flash Attention) enabled")
        model.eval()
        EMBEDDER_TYPE = 'colbert'
        print(f'✓ Model Loaded Successfully on {device}')
    except Exception as e:
        print(f'ColBERT failed: {e}')
        print('Falling back to SBERT')
        USE_COLBERT = False

if not USE_COLBERT:
    model = SentenceTransformer('all-MiniLM-L6-v2')
    EMBEDDER_TYPE = 'sbert'
    print('Using SBERT: all-MiniLM-L6-v2')

In [ ]:
def embed_sbert(texts, batch_size=512):
    embeddings = model.encode(
        texts,
        batch_size=batch_size,
        show_progress_bar=True,
        convert_to_numpy=True,
        normalize_embeddings=True
    )
    return embeddings.astype(np.float16)

def embed_colbert(texts, start_batch_size=16384):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    sort_idx = sorted(range(len(texts)), key=lambda i: len(texts[i]))
    sorted_texts = [texts[i] for i in sort_idx]
    unsort_idx = np.argsort(sort_idx)

    print("    Pre-tokenizing...")
    encodings = tokenizer(
        sorted_texts,
        padding=False,
        truncation=True,
        max_length=180,
    )
    del sorted_texts

    all_embeddings = []
    i = 0
    bs = start_batch_size
    total = len(texts)
    pbar = tqdm(total=total, unit="txt")

    while i < total:
        batch_enc = {
            'input_ids': encodings['input_ids'][i:i + bs],
            'attention_mask': encodings['attention_mask'][i:i + bs],
        }

        try:
            inputs = tokenizer.pad(batch_enc, padding='longest', return_tensors='pt').to(device)

            with torch.no_grad():
                outputs = model(**inputs)
                token_emb = outputs.last_hidden_state
                mask = inputs['attention_mask']

                mask_expanded = mask.unsqueeze(-1)
                sum_emb = (token_emb * mask_expanded).sum(dim=1)
                count = mask_expanded.sum(dim=1)
                pooled = sum_emb / count
                pooled = pooled / (pooled.norm(dim=1, keepdim=True) + 1e-9)

                all_embeddings.append(pooled.float().cpu().numpy())

            done = min(bs, total - i)
            pbar.update(done)
            i += done

        except torch.cuda.OutOfMemoryError:
            torch.cuda.empty_cache()
            gc.collect()
            bs = max(512, bs // 2)
            pbar.set_postfix(bs=bs)

    pbar.close()
    del encodings

    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    embeddings = np.vstack(all_embeddings).astype(np.float16)
    return embeddings[unsort_idx]

In [ ]:
def process_year(year):
    data_path = READY_DIR / f'year={year}' / 'data.parquet'
    if not data_path.exists():
        print(f'{year}: Not found at {data_path}')
        return

    emb_path = EMBEDDINGS_DIR / f'embeddings_{year}.npy'
    ids_path = EMBEDDINGS_DIR / f'ids_{year}.parquet'

    if emb_path.exists() and ids_path.exists():
        print(f'{year}: Already complete, skipping')
        return

    print(f'\nProcessing {year}...')
    df = pl.read_parquet(data_path)
    texts = df['doc_text_music'].fill_null('').to_list()
    total = len(texts)
    print(f'  Rows: {total:,} | Embedder: {EMBEDDER_TYPE}')

    chunk_dir = EMBEDDINGS_DIR / f'chunks_{year}'
    chunk_dir.mkdir(exist_ok=True)
    n_chunks = (total + EMBED_CHUNK - 1) // EMBED_CHUNK

    for c in range(n_chunks):
        chunk_path = chunk_dir / f'chunk_{c}.npy'
        if chunk_path.exists():
            print(f'  Chunk {c+1}/{n_chunks}: already saved, skipping')
            continue

        start = c * EMBED_CHUNK
        end = min(start + EMBED_CHUNK, total)
        batch_texts = texts[start:end]

        print(f'  Chunk {c+1}/{n_chunks}: embedding {len(batch_texts):,} texts...')
        if EMBEDDER_TYPE == 'colbert':
            emb = embed_colbert(batch_texts)
        else:
            emb = embed_sbert(batch_texts)

        np.save(chunk_path, emb)
        del emb
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()

    print(f'  Combining {n_chunks} chunks...')
    chunks = [np.load(chunk_dir / f'chunk_{c}.npy') for c in range(n_chunks)]
    embeddings = np.vstack(chunks)
    del chunks
    gc.collect()

    np.save(emb_path, embeddings)
    print(f'  Saved: {emb_path} | Shape: {embeddings.shape}')

    id_col = 'track_id' if 'track_id' in df.columns else 'permalink_url'
    ids_df = df.select([id_col]).rename({id_col: 'track_id'})
    ids_df.write_parquet(ids_path)
    print(f'  Saved: {ids_path} (using {id_col})')

    del embeddings, df, texts
    gc.collect()

    shutil.rmtree(chunk_dir)
    print(f'  Cleaned up temp chunks')

In [ ]:
for year in YEAR_RANGE:
    process_year(year)

print('\n' + '=' * 50)
print('EMBEDDING COMPLETE')
print('=' * 50)

for f in sorted(EMBEDDINGS_DIR.glob('*.npy')):
    emb = np.load(f)
    print(f'{f.name}: {emb.shape}')

In [ ]:
import subprocess

zip_path = "/content/embeddings.zip"
subprocess.run(["zip", "-r", zip_path, str(EMBEDDINGS_DIR)], check=True)

zip_size = Path(zip_path).stat().st_size / (1024**3)
print(f"Zipped: {zip_size:.1f} GB")

try:
    from google.colab import files
    print("Downloading... (check your browser downloads)")
    files.download(zip_path)
except ImportError:
    print(f"Not on Colab. Zip saved at: {zip_path}")